# LiDAR Uncertainty: A Comparison of Two California Sites.

## The Soap Site
![image.png](attachment:image.png)
image credit: National Ecological Observation Network

In [13]:
## The SJER Site
![image.png](attachment:image.png)
image credit: National Ecological Observation Network

SyntaxError: invalid syntax (1443426053.py, line 3)

In [2]:
# Import necessary packages
import os
import pathlib

import earthpy as et
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterstats as rs
import rioxarray as rxr
import seaborn as sns
import xarray as xr

# Download data from earth py
et.data.get_data('spatial-vector-lidar')

# General housekeeping
home_dir = os.path.join(pathlib.Path.home(), 'earth-analytics',
                        'data', 'spatial-vector-lidar')
os.chdir(home_dir)

# If the dir does not exist, create it
output_path = os.path.join(home_dir,
                           "outputs")

In [3]:
class NEONDataLoader:

    '''This class creates all of our necessary paths for downloads

    parameters:
    ----------
    site_name = str
    id_col = str
    formatting_dict = key (?)

    returns:
    -------
    Data is opened and transferred to correct locations.
    '''

    # Create necessary variables for SOAP and SJER sites.
    base_dir_tmpl = os.path.join('california',
                                 'neon-{site_name_low}-site')

    insitu_path_tmpl = os.path.join('{base_dir}', '2013',
                                    'insitu', 'veg{separator}structure',
                                    'D17_2013_{site_name_up}_vegStr.csv')
    plots_path_tmpl = os.path.join(
        '{base_dir}', 'vector_data', '{site_name_up}{plot}_centroids.shp')
    chm_path_tmpl = os.path.join(
        '{base_dir}', '2013', 'lidar', '{site_name_up}_lidarCHM.tif')

    # Create init function download both sets of data.
    def __init__(self, site_name, id_col, formatting_dict={}):
        self.site_name = site_name
        self.id_col = id_col
        self.formatting_dict = formatting_dict
        self.formatting_dict['site_name_low'] = site_name.lower()
        self.formatting_dict['site_name_up'] = site_name.upper()
        self.formatting_dict['base_dir'] = self.base_dir_tmpl.format(
            **self.formatting_dict)

        self.insitu_path = self.insitu_path_tmpl.format(**self.formatting_dict)
        self.plots_path = self.plots_path_tmpl.format(**self.formatting_dict)
        self.chm_path = self.chm_path_tmpl.format(**self.formatting_dict)

In [4]:
sjer_data_loader = NEONDataLoader('SJER', 'Plot_ID',
                                  {'separator': '_', 'plot': '_plot'})
sjer_data_loader

In [5]:
soap_data_loader=NEONDataLoader('SOAP', 'ID', 
                                {'separator':'-', 'plot':''})

soap_data_loader.insitu_path

'california\\neon-soap-site\\2013\\insitu\\veg-structure\\D17_2013_SOAP_vegStr.csv'

In [91]:
class AnalyzeLidarUncertainty:

    '''This class calculates stats for lidar and insitu and merges the two data frames.

        parameters:
        ----------
        lidar_path: str (or path?) 
        insitu_path: str
        plots_path: str

        returns:
        -------
        Processed data frame.

        '''

    def __init__(self, lidar_path, insitu_path, plots_path, study_site):
        self.lidar_path = lidar_path
        self.insitu_path = insitu_path
        self.plots_path = plots_path
        self.study_site = study_site

    def calc_lidar_chm_stats(self):
        '''
        Calculates lidar mean and max tree heights.
        '''

        lidar_chm = rxr.open_rasterio(
            self.lidar_path, masked=True).squeeze()
        lidar_chm_clean = lidar_chm.where(lidar_chm > 0, np.nan)

        plots_gdf = gpd.read_file(self.plots_path)
        plots_gdf.geometry = plots_gdf.geometry.buffer(20)

        lidar_chm_stats = rs.zonal_stats(
            plots_gdf,
            lidar_chm_clean.values,
            affine=lidar_chm_clean.rio.transform(),
            nodata=-999,
            geojson_out=True,
            copy_properties=True,
            stats='count mean max')

        # Create a gdf based on the stats requested from zonal stats and rename columns.
        lidar_chm_gdf = gpd.GeoDataFrame.from_features(lidar_chm_stats)

        lidar_chm_gdf.rename(
            columns={'max': 'lidar_max',
                     'mean': 'lidar_mean',
                     'ID': 'plotid',
                     'Plot_ID': 'plotid'}, inplace=True)

        return lidar_chm_gdf

    def calc_insitu_dfs(self):
        '''
        Calculate insitu tree height mean and max.
        '''

        df = (pd.read_csv(self.insitu_path)
              .groupby('plotid')
              .stemheight
              .agg(['max', 'mean'])
              .rename(columns={'max': 'insitu_max',
                               'mean': 'insitu_mean',
                               'ID': 'plotid',
                               'Plot_ID': 'plotid'}))

        return df

    def merge_dfs(self):
        '''
        Merges insitu and lidar data frames together.

        '''

        chm_stats_gdf = self.calc_lidar_chm_stats()
        insitu_height_df = self.calc_insitu_dfs()

        if (self.study_site == 'soap'):
            ss = self.study_site.upper()
            chm_stats_gdf['plotid'] = ss + chm_stats_gdf['plotid']

        data_frame = chm_stats_gdf.merge(
            insitu_height_df, right_on='plotid', left_on='plotid')
        return data_frame


sjer_site = AnalyzeLidarUncertainty(
    sjer_data_loader.chm_path, sjer_data_loader.insitu_path,
    sjer_data_loader.plots_path, sjer_data_loader.formatting_dict['site_name_low'])

sjer_site.calc_lidar_chm_stats()

,geometry,plotid,Point,northing,easting,plot_type,lidar_max,lidar_mean,count
0,"POLYGON ((255872.376 4111567.818, 255872.280 4...",SJER1068,center,4111567.818,255852.376,trees,19.049999,11.544347,161
1,"POLYGON ((257426.967 4111298.971, 257426.871 4...",SJER112,center,4111298.971,257406.967,trees,24.019999,10.369277,443
2,"POLYGON ((256858.760 4110819.876, 256858.664 4...",SJER116,center,4110819.876,256838.760,grass,16.070000,7.518398,643
3,"POLYGON ((256196.947 4108752.026, 256196.851 4...",SJER117,center,4108752.026,256176.947,trees,11.059999,7.675346,245
4,"POLYGON ((255988.372 4110476.079, 255988.276 4...",SJER120,center,4110476.079,255968.372,grass,5.740000,4.591177,17
5,"POLYGON ((257098.867 4111388.570, 257098.771 4...",SJER128,center,4111388.570,257078.867,trees,19.139999,8.987087,381
6,"POLYGON ((256703.434 4111071.087, 256703.338 4...",SJER192,center,4111071.087,256683.434,grass,16.549999,7.229096,929
7,"POLYGON ((256737.467 4112167.778, 256737.371 4...",SJER272,center,4112167.778,256717.467,trees,11.840000,7.107061,711
8,"POLYGON ((256054.390 4111533.879, 256054.294 4...",SJER2796,center,4111533.879,256034.390,soil,20.279999,6.409630,270
9,"POLYGON ((258517.102 4109856.983, 258517.006 4...",SJER3239,center,4109856.983,258497.102,soil,12.910000,6.009128,195


In [84]:
soap_site = AnalyzeLidarUncertainty(soap_data_loader.chm_path, soap_data_loader.insitu_path,
                                    soap_data_loader.plots_path, soap_data_loader.formatting_dict['site_name_low'])
soap_site.calc_lidar_chm_stats()

,geometry,OBJECTID,plotid,plotType,subtype,plotDimens,Protocols,lidar_max,lidar_mean,count
0,"POLYGON ((297815.197 4101493.028, 297815.101 4...",1,27,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",50.250000,20.455827,822
1,"POLYGON ((297065.197 4100713.028, 297065.101 4...",2,43,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",54.099998,19.804080,1000
2,"POLYGON ((299825.197 4101013.028, 299825.101 4...",3,63,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",32.480000,16.989834,1082
3,"POLYGON ((298175.197 4101433.028, 298175.101 4...",4,91,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",50.309998,21.980976,1075
4,"POLYGON ((298715.197 4100833.028, 298715.101 4...",5,95,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",33.869999,17.412048,1138
5,"POLYGON ((297065.197 4100083.028, 297065.101 4...",6,139,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",49.919998,19.417840,1094
6,"POLYGON ((299885.197 4100413.028, 299885.101 4...",7,143,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",27.340000,10.454653,778
7,"POLYGON ((297545.197 4100533.028, 297545.101 4...",8,187,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",47.959999,22.895033,1035
8,"POLYGON ((298775.197 4101403.028, 298775.101 4...",9,287,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",53.969997,17.602865,1204
9,"POLYGON ((297515.197 4100683.028, 297515.101 4...",10,299,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",34.189999,18.852112,1113


In [85]:
sjer_site.calc_insitu_dfs()

,insitu_max,insitu_mean
plotid,,
SJER1068,19.3,3.866667
SJER112,23.9,8.221429
SJER116,16.0,8.218750
SJER117,11.0,6.512500
SJER120,8.8,7.600000
SJER128,18.2,5.211765
SJER192,13.7,6.769565
SJER272,12.4,6.819048
SJER2796,9.4,5.085714


In [86]:
soap_site.calc_insitu_dfs()

,insitu_max,insitu_mean
plotid,,
SOAP1343,6.5,2.112500
SOAP139,120.0,4.745902
SOAP143,19.7,2.530702
SOAP1515,3.4,1.776923
SOAP1563,1.9,1.071429
SOAP1611,4.2,1.720000
SOAP1695,17.0,3.056604
SOAP187,134.0,4.878947
SOAP223,8.2,2.444186


In [87]:
soap_site.merge_dfs()

,geometry,OBJECTID,plotid,plotType,subtype,plotDimens,Protocols,lidar_max,lidar_mean,count,insitu_max,insitu_mean
0,"POLYGON ((297065.197 4100713.028, 297065.101 4...",2,SOAP43,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",54.099998,19.804080,1000,51.1,5.047000
1,"POLYGON ((299825.197 4101013.028, 299825.101 4...",3,SOAP63,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",32.480000,16.989834,1082,33.0,9.232787
2,"POLYGON ((298715.197 4100833.028, 298715.101 4...",5,SOAP95,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",33.869999,17.412048,1138,28.1,6.050943
3,"POLYGON ((297065.197 4100083.028, 297065.101 4...",6,SOAP139,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",49.919998,19.417840,1094,120.0,4.745902
4,"POLYGON ((299885.197 4100413.028, 299885.101 4...",7,SOAP143,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",27.340000,10.454653,778,19.7,2.530702
5,"POLYGON ((297545.197 4100533.028, 297545.101 4...",8,SOAP187,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",47.959999,22.895033,1035,134.0,4.878947
6,"POLYGON ((297515.197 4100683.028, 297515.101 4...",10,SOAP299,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",34.189999,18.852112,1113,33.1,13.896154
7,"POLYGON ((297065.197 4100593.028, 297065.101 4...",11,SOAP331,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",45.500000,18.149803,907,39.2,5.009848
8,"POLYGON ((299315.197 4101463.028, 299315.101 4...",23,SOAP223,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",13.469999,6.155488,410,8.2,2.444186
9,"POLYGON ((297785.197 4101193.028, 297785.101 4...",24,SOAP283,distributed,base,40.0,"beetles, soils, coarse-downed wood, leaf area ...",19.369999,5.781789,475,13.8,3.328571


In [88]:
sjer_site.merge_dfs()

,geometry,plotid,Point,northing,easting,plot_type,lidar_max,lidar_mean,count,insitu_max,insitu_mean
0,"POLYGON ((255872.376 4111567.818, 255872.280 4...",SJER1068,center,4111567.818,255852.376,trees,19.049999,11.544347,161,19.3,3.866667
1,"POLYGON ((257426.967 4111298.971, 257426.871 4...",SJER112,center,4111298.971,257406.967,trees,24.019999,10.369277,443,23.9,8.221429
2,"POLYGON ((256858.760 4110819.876, 256858.664 4...",SJER116,center,4110819.876,256838.760,grass,16.070000,7.518398,643,16.0,8.218750
3,"POLYGON ((256196.947 4108752.026, 256196.851 4...",SJER117,center,4108752.026,256176.947,trees,11.059999,7.675346,245,11.0,6.512500
4,"POLYGON ((255988.372 4110476.079, 255988.276 4...",SJER120,center,4110476.079,255968.372,grass,5.740000,4.591177,17,8.8,7.600000
5,"POLYGON ((257098.867 4111388.570, 257098.771 4...",SJER128,center,4111388.570,257078.867,trees,19.139999,8.987087,381,18.2,5.211765
6,"POLYGON ((256703.434 4111071.087, 256703.338 4...",SJER192,center,4111071.087,256683.434,grass,16.549999,7.229096,929,13.7,6.769565
7,"POLYGON ((256737.467 4112167.778, 256737.371 4...",SJER272,center,4112167.778,256717.467,trees,11.840000,7.107061,711,12.4,6.819048
8,"POLYGON ((256054.390 4111533.879, 256054.294 4...",SJER2796,center,4111533.879,256034.390,soil,20.279999,6.409630,270,9.4,5.085714
9,"POLYGON ((258517.102 4109856.983, 258517.006 4...",SJER3239,center,4109856.983,258497.102,soil,12.910000,6.009128,195,17.9,3.920833


In [90]:
def plot_regression(lidar_mean, insitu_mean, lidar_max, insitu_max):

    # Plot SJER data with regression lines/scatterplot.
    Title= {'SJER', 'SOAP'}
    fig, (ax1, ax2)=plt.subplots(1,2, figsize=(12,12))
    fig.suptitle(
        '{} Insitu & Lidar Comparison', fontsize=14)
    ax1.scatter(df.lidar_mean, df.insitu_mean)
    ax1.set(xlim=(0,30), ylim=(0,30), aspect='equal', title='Mean Values')
    sns.regplot('lidar_mean', 'insitu_mean',
               data=df,
               color='purple',
               ax=ax1)
    ax1.plot((0,1),(0,1), transform=ax1.transAxes, ls='--', c='k')

    ax2.scatter(df.lidar_max, df.insitu_max)
    ax2.set(xlim=(0,30), ylim=(0,30), aspect='equal', title='Max Values')
    sns.regplot('lidar_max', 'insitu_max', 
                data=df, 
                color='purple', 
                ax=ax2)
    ax2.plot((0,1),(0,1), transform=ax2.transAxes, ls='--', c='k')
    return regression_plot

plot_regression(soap_site.merge_dfs(lidar_mean, insitu_mean, lidar_max, insitu_max))

NameError: name 'lidar_mean' is not defined